In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset_completo = pd.read_csv("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/casos-covid-19/casos_covid19.csv")

C:\Users\44594722\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [114]:
meses_replace = {"JAN":"01","FEB":"02","MAR":"03","APR":"04","MAY":"05","JUN":"06","JUL":"07","AUG":"08","SEP":"09","OCT":"10","NOV":"11","DEC":"12"}

In [115]:
dataset_completo_procesado = dataset_completo.copy()
####################################### CREAMOS LA COLUMNA DEL DIA DE CLASIFICACION ########################################
mes_clasificacion = dataset_completo_procesado["fecha_clasificacion"].str[2:5].replace(meses_replace)
dia_clasificacion = dataset_completo_procesado["fecha_clasificacion"].str[:2]
dataset_completo_procesado["mes/dia"] = mes_clasificacion.map(str) +"/"+ dia_clasificacion.map(str)
#################################################################################################################

################################## CREAMOS LA COLUMNA DEL DIA DE LA MUERTE ##################################################
mes_muerte = dataset_completo_procesado["fecha_fallecimiento"].str[2:5].replace(meses_replace)
dia_muerte = dataset_completo_procesado["fecha_fallecimiento"].str[:2]
dataset_completo_procesado["mes/dia_fallecimiento"] = mes_muerte.map(str) +"/"+ dia_muerte.map(str)
#############################################################################################################################

In [116]:
#################################### AQUI UTILIZAREMOS DISTINTOS CRITERIOS PARA SORTEAR EL DATASET ##########################
dataset_completo_procesado_sorted_clasificacion = dataset_completo_procesado.copy().sort_values("mes/dia")
dataset_completo_procesado_sorted_muerte = dataset_completo_procesado.copy().sort_values("mes/dia_fallecimiento")

In [117]:
def nuevos_casos_caba_dia(dia, residentes = True):
    filtro = dataset_completo_procesado.copy()
    if residentes:
        filtro = filtro[filtro["provincia"]=="CABA"]
    confirmados = filtro[filtro["clasificacion"]=="confirmado"]
    cant_casos = confirmados[confirmados["mes/dia"]==dia]
    return {dia: len(cant_casos)}

In [118]:
def nuevas_muertes_caba_dia(dia, residentes = True):
    filtro = dataset_completo_procesado.copy()
    if residentes:
        filtro = filtro[filtro["provincia"]=="CABA"]
    muertes = filtro[filtro["fallecido"]=="si"]
    cant_muertes = muertes[muertes["mes/dia_fallecimiento"]==dia]
    return {dia: len(cant_muertes)}

In [119]:
def nuevos_casos_caba_rango(desde="0",hasta="3000", residentes = True):
    filtro = dataset_completo_procesado_sorted_clasificacion.copy()
    if residentes:
        filtro = filtro[filtro["provincia"]=="CABA"]
    confirmados = filtro[filtro["clasificacion"]=="confirmado"]
    confirmados_dict = {}
    for i in confirmados["mes/dia"]:
        if desde <= i < hasta:
            confirmados_dict[i] = confirmados_dict.get(i,0) + 1 
    return confirmados_dict

In [120]:
def nuevas_muertes_caba_rango(desde="0",hasta="3000", residentes = True):
    filtro = dataset_completo_procesado_sorted_muerte.copy()
    if residentes:
        filtro = filtro[filtro["provincia"]=="CABA"]
    muertes = filtro[filtro["fallecido"]=="si"]
    muertes_dict = {}
    for i in muertes["mes/dia_fallecimiento"]:
        if desde <= i < hasta:
            muertes_dict[i] = muertes_dict.get(i,0) + 1 
    return muertes_dict

## Lo que viene a continuacion son pruebas

In [121]:
nuevos_casos_caba_dia("11/15")

{'11/15': 32}

In [122]:
nuevas_muertes_caba_rango("11/10")

{'11/10': 15, '11/11': 9, '11/12': 6, '11/13': 1, '11/14': 4, '11/15': 1}

In [123]:
nuevos_casos_caba_rango("11/12")

{'11/12': 426, '11/13': 375, '11/14': 267, '11/15': 32}

In [128]:
testeos_dia = dataset_completo[dataset_completo["fecha_clasificacion"]=="12NOV2020:00:00:00.000000"]

In [129]:
confirmados_dia = testeos_dia[testeos_dia["clasificacion"]=="confirmado"]

In [130]:
len(confirmados_dia[confirmados_dia["provincia"]=="CABA"])

426

In [131]:
muertes_dia = dataset_completo[dataset_completo["fecha_fallecimiento"]=="12NOV2020:00:00:00.000000"]

In [133]:
len(muertes_dia[muertes_dia["provincia"]=="CABA"])

6